In [ ]:
all_messages_per_topic = intents_df.groupby(['topic'], as_index=False).agg({'message': ' '.join})

In [ ]:
def c_tf_idf(documents, m, ngram_range=(1,1)):
    count = CountVectorizer(ngram_range = ngram_range, stop_words = 'english').fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)
    
    return tf_idf, count

def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=10):
    words = count.get_feature_names()
    labels = list(docs_per_topic.topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    top_n_words_only = {label: [words[j] for j in indices[i]][::-1] for i, label in enumerate(labels)}
    
    return top_n_words, top_n_words_only

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['topic']).message.count()
                   .reset_index()
                   .rename({'message':'size'}, axis='columns')
                   .sort_values('size', ascending=False))
    return topic_sizes

In [ ]:
tf_idf, count = c_tf_idf(all_messages_per_topic.message.values, m = len(data_sample))

In [ ]:
top_n_words, top_n_words_only = extract_top_n_words_per_topic(tf_idf, count, all_messages_per_topic, n=10)

In [ ]:
top_n_words[0]

In [ ]:
top_n_words[-1]

In [ ]:
df_keywords = pd.DataFrame()
df_keywords['topic'] = top_n_words_only.keys()
df_keywords['keywords'] = top_n_words_only.values()

In [ ]:
extract_topic_sizes(intents_df).merge(df_keywords, how='left', on='topic')

In [ ]:
intents_df[intents_df['topic']==1]

In [ ]:
intents_df[intents_df['message'].str.contains('card')]

In [ ]:
umap_data = umap.UMAP(n_neighbors=15, 
                      n_components=2, 
                      min_dist = 0.0, 
                      metric='cosine').fit_transform(use_embeddings)

result = pd.DataFrame(umap_data, columns=['x', 'y'])
result['labels'] = clusters.labels_

#visualizing clusters
fig, ax = plt.subplots(figsize=(20, 10))
outliers = result.loc[result.labels == -1, :]
clustered = result.loc[result.labels != -1, :]
plt.scatter(outliers.x, outliers.y, color = '#BDBDBD', s=0.8)
plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=0.8, cmap='hsv_r')
plt.colorbar()